In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/CS5246/AITA')

Mounted at /content/drive


In [2]:
!pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 10.4 MB/s eta 0:00:00


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import re
import string
import contractions

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import networkx as nx

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import f1_score

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
def get_sentences(text):
    # Tokenize the text into sentences
    return len(sent_tokenize(text))

porter_stemmer = PorterStemmer()
wordnet_lemmatizer = WordNetLemmatizer()

def tokenize_and_stem(text):
    expanded_text = contractions.fix(text)
    clean_text = re.sub(r'http[s]?://\S+', '', expanded_text)
    tokens = word_tokenize(clean_text)
    lemmas = [wordnet_lemmatizer.lemmatize(token) for token in tokens]
    return lemmas

### Load data

In [5]:
train = pd.read_csv('train_summarised_gpt2.csv')
test = pd.read_csv('test_summarised_gpt2.csv')
df = pd.concat([train, test], axis=0)
df.head()

,Unnamed: 0,flag,content,summarised
0,0,0,My bf (21) and me (22) have been together for ...,My bf (21) and me (22) have been together for ...
1,1,0,Yesterday afternoon my(19m) aunt's boyfriend b...,Yesterday afternoon my(19m) aunt's boyfriend b...
2,2,0,So I've been dating this girl since early dece...,On Tuesday I told her about some new things I ...
3,3,0,I like to send my roommate pictures of cute th...,I like to send my roommate pictures of cute th...
4,4,0,"Good Afternoon, I'm a 29yo US Venezuelan immig...","Good Afternoon, I'm a 29yo US Venezuelan immig..."


In [8]:
df['sent_length'] = df['summarised'].apply(get_sentences)
df['sent_length'].describe()

count    16872.000000
mean         4.090801
std          1.804454
min          1.000000
25%          3.000000
50%          4.000000
75%          5.000000
max         21.000000
Name: sent_length, dtype: float64

#2 part logistic regression

###Preprocessing

In [13]:
def split_to_columns(df):
  sent1 = []
  sent2 = []
  sent3 = []
  sent4 = []
  sent5 = []

  for text in df['summarised']:
    length = len(sent_tokenize(text))
    sentences = sent_tokenize(text)
    while length < 5:
      sentences.append('')
      length += 1
    sent1.append(sentences[0])
    sent2.append(sentences[1])
    sent3.append(sentences[2])
    sent4.append(sentences[3])
    sent5.append(sentences[4])

  df['sent1'] = sent1
  df['sent2'] = sent2
  df['sent3'] = sent3
  df['sent4'] = sent4
  df['sent5'] = sent5

  return df

In [14]:
train = split_to_columns(train)
test = split_to_columns(test)

In [10]:
def flatten(data):
    results = []
    for index, row in data.iterrows():
      results.extend(row)
    return results

In [15]:
#flatten the data so that each sentence is now 1 data point
X_train_flattened = flatten(train[['sent1', 'sent2', 'sent3', 'sent4', 'sent5']].fillna(''))
X_test_flattened = flatten(test[['sent1', 'sent2', 'sent3', 'sent4', 'sent5']].fillna(''))

In [16]:
#Vectorise the sentences
vectorizer = TfidfVectorizer(tokenizer=tokenize_and_stem, token_pattern=None)

vectorizer.fit(X_train_flattened)
X_train_vector = vectorizer.transform(X_train_flattened)
X_test_vector = vectorizer.transform(X_test_flattened)

In [17]:
#Repeat each label 5 times to match the new flattened data
y_train_flattened = np.repeat(train['flag'], 5)
y_test_flattened = np.repeat(test['flag'], 5)

### Logistic Regression

In [18]:
#Stage 1 Logistic Regression, to train the output for each sentence
stage_1 = LogisticRegression(solver="saga")
stage_1.fit(X_train_vector, y_train_flattened)
X_train_intermediate_results = stage_1.predict(X_train_vector)
X_test_intermediate_results = stage_1.predict(X_test_vector)

In [19]:
#reshape to push into stage 2 logistic regression
X_train_intermediate_input = X_train_intermediate_results.reshape(int(len(X_train_intermediate_results)/5), 5)
X_test_intermediate_input = X_test_intermediate_results.reshape(int(len(X_test_intermediate_results)/5), 5)
print(X_train_intermediate_input.shape)
print(X_test_intermediate_input.shape)

(10872, 5)
(6000, 5)


In [20]:
#Stage 2 Logistic Regression training score
cross_val_scores = cross_val_score(LogisticRegression(solver="saga"), X_train_intermediate_input, train['flag'], cv=10, scoring="f1")
print(f"cross-validation f1 scores:", cross_val_scores)
print(f"average cross-validation f1 score:", cross_val_scores.mean())

cross-validation f1 scores: [0.92476754 0.92216582 0.89477912 0.90375204 0.81740614 0.79500891
 0.76687669 0.66209598 0.63329928 0.64736298]
average cross-validation f1 score: 0.7967514507679276


In [21]:
#Stage 2 Logistic Regression training score
cross_val_scores = cross_val_score(LogisticRegression(solver="saga"), X_train_intermediate_input, train['flag'], cv=10, scoring="accuracy")
print(f"cross-validation accuracy:", cross_val_scores)
print(f"average cross-validation accuracy:", cross_val_scores.mean())

cross-validation accuracy: [0.91819853 0.91544118 0.87948482 0.89144434 0.80312787 0.78840846
 0.76172953 0.6826127  0.66973321 0.68629255]
average cross-validation accuracy: 0.7996473192542887


In [22]:
#Stage 2 Logistic Regression testing score
stage_2 = LogisticRegression(solver="saga")
stage_2.fit(X_train_intermediate_input, train['flag'])
y_pred = stage_2.predict(X_test_intermediate_input)
print(f"accuracy:", metrics.accuracy_score(test['flag'], y_pred))
print(f"f1-score:", metrics.f1_score(test['flag'], y_pred))

accuracy: 0.7516666666666667
f1-score: 0.547112462006079


In [23]:
#Check whether majority polling gives the same results
y_test = [round(np.mean(x)) for x in X_test_intermediate_input]
print(f"accuracy:", metrics.accuracy_score(test['flag'], y_test))
print(f"f1-score:", metrics.f1_score(test['flag'], y_test ))

accuracy: 0.5946666666666667
f1-score: 0.4099951479864143


### NBaive Bayes

In [24]:
#Stage 1 NB, to train the output for each sentence
stage_1 = MultinomialNB()
stage_1.fit(X_train_vector, y_train_flattened)
X_train_intermediate_results = stage_1.predict(X_train_vector)
X_test_intermediate_results = stage_1.predict(X_test_vector)

In [25]:
#reshape to push into stage 2 NB
X_train_intermediate_input = X_train_intermediate_results.reshape(int(len(X_train_intermediate_results)/5), 5)
X_test_intermediate_input = X_test_intermediate_results.reshape(int(len(X_test_intermediate_results)/5), 5)
print(X_train_intermediate_input.shape)
print(X_test_intermediate_input.shape)

(10872, 5)
(6000, 5)


In [26]:
#Stage 2 NB training score
cross_val_scores = cross_val_score(MultinomialNB(), X_train_intermediate_input, train['flag'], cv=10, scoring="f1")
print(f"cross-validation f1 scores:", cross_val_scores)
print(f"average cross-validation f1 score:", cross_val_scores.mean())

cross-validation f1 scores: [0.8377193  0.83430233 0.8398524  0.83994126 0.73988439 0.6731794
 0.64       0.62401402 0.57297297 0.58152174]
average cross-validation f1 score: 0.7183387809246129


In [27]:
#Stage 2 Logistic Regression training score
cross_val_scores = cross_val_score(MultinomialNB(), X_train_intermediate_input, train['flag'], cv=10, scoring="accuracy")
print(f"cross-validation accuracy:", cross_val_scores)
print(f"average cross-validation accuracy:", cross_val_scores.mean())

cross-validation accuracy: [0.79595588 0.79044118 0.80036799 0.79944802 0.71021159 0.66145354
 0.61913523 0.60533579 0.56393744 0.574977  ]
average cross-validation accuracy: 0.6921263664159316


In [28]:
#Stage 2 Logistic Regression testing score
stage_2 = MultinomialNB()
stage_2.fit(X_train_intermediate_input, train['flag'])
y_pred = stage_2.predict(X_test_intermediate_input)
print(f"accuracy:", metrics.accuracy_score(test['flag'], y_pred))
print(f"f1-score:", metrics.f1_score(test['flag'], y_pred))

accuracy: 0.5826666666666667
f1-score: 0.44306049822064064


In [29]:
#Check whether majority polling gives the same results
y_test = [round(np.mean(x)) for x in X_test_intermediate_input]
print(f"accuracy:", metrics.accuracy_score(test['flag'], y_test))
print(f"f1-score:", metrics.f1_score(test['flag'], y_test ))

accuracy: 0.4698333333333333
f1-score: 0.3682224428997021


### Mixed

In [30]:
#Stage 1 NB, to train the output for each sentence
stage_1 = MultinomialNB()
stage_1.fit(X_train_vector, y_train_flattened)
X_train_intermediate_results = stage_1.predict(X_train_vector)
X_test_intermediate_results = stage_1.predict(X_test_vector)

In [31]:
#reshape to push into stage 2 logistic regression
X_train_intermediate_input = X_train_intermediate_results.reshape(int(len(X_train_intermediate_results)/5), 5)
X_test_intermediate_input = X_test_intermediate_results.reshape(int(len(X_test_intermediate_results)/5), 5)
print(X_train_intermediate_input.shape)
print(X_test_intermediate_input.shape)

(10872, 5)
(6000, 5)


In [32]:
#Stage 2 Logistic Regression training score
cross_val_scores = cross_val_score(LogisticRegression(solver="saga"), X_train_intermediate_input, train['flag'], cv=10, scoring="f1")
print(f"cross-validation f1 scores:", cross_val_scores)
print(f"average cross-validation f1 score:", cross_val_scores.mean())

cross-validation f1 scores: [0.92320396 0.91451613 0.90731707 0.90634921 0.84552846 0.83417085
 0.80675105 0.73122172 0.72576177 0.72268908]
average cross-validation f1 score: 0.8317509304567269


In [33]:
#Stage 2 Logistic Regression training score
cross_val_scores = cross_val_score(LogisticRegression(solver="saga"), X_train_intermediate_input, train['flag'], cv=10, scoring="accuracy")
print(f"cross-validation accuracy:", cross_val_scores)
print(f"average cross-validation accuracy:", cross_val_scores.mean())

cross-validation accuracy: [0.91452206 0.90257353 0.8951242  0.89144434 0.82520699 0.81784729
 0.78932843 0.72677093 0.72677093 0.72677093]
average cross-validation accuracy: 0.8216359617674117


In [34]:
#Stage 2 Logistic Regression testing score
stage_2 = LogisticRegression(solver="saga")
stage_2.fit(X_train_intermediate_input, train['flag'])
y_pred = stage_2.predict(X_test_intermediate_input)
print(f"accuracy:", metrics.accuracy_score(test['flag'], y_pred))
print(f"f1-score:", metrics.f1_score(test['flag'], y_pred))

accuracy: 0.6605
f1-score: 0.47567567567567565


In [35]:
#Check whether majority polling gives the same results
y_test = [round(np.mean(x)) for x in X_test_intermediate_input]
print(f"accuracy:", metrics.accuracy_score(test['flag'], y_test))
print(f"f1-score:", metrics.f1_score(test['flag'], y_test ))

accuracy: 0.4698333333333333
f1-score: 0.3682224428997021


#Justice model

In [ ]:
justice = pd.read_csv('/content/drive/MyDrive/CS5246/AITA/justice_test.csv')
justice.head()

,label,scenario
0,1,I used to walk around the park with my coworke...
1,0,I used to walk around the park with my coworke...
2,1,I used to walk around the park with my coworke...
3,0,I used to walk around the park with my coworke...
4,0,I usually give a gift to my girlfriend on Vale...


In [ ]:
#Creating Training and Testing data, and vectorise
X_train, X_test, y_train, y_test = train_test_split(justice['scenario'], justice['label'], test_size=0.2, random_state=42)

vectorizer = TfidfVectorizer(tokenizer=tokenize_and_stem, token_pattern=None)
vectorizer.fit(X_train)
X_train_vector = vectorizer.transform(X_train)
X_test_vector = vectorizer.transform(X_test)

In [ ]:
#Logistic Regression
cross_val_scores = cross_val_score(LogisticRegression(solver="saga"), X_train_vector, y_train, cv=5, scoring="f1")
print(f"cross-validation scores:", cross_val_scores)
print(f"average cross-validation score:", cross_val_scores.mean())

justice_model = LogisticRegression(solver="saga")
justice_model.fit(X_train_vector, y_train)
y_pred_train = justice_model.predict(X_train_vector)
y_pred_test = justice_model.predict(X_test_vector)

f1_train = f1_score(y_train, y_pred_train)
f1_test = f1_score(y_test, y_pred_test)

print("F1 train score:", f1_train)
print("F1 test score:", f1_test)

cross-validation scores: [0.43378995 0.46046512 0.49433107 0.45433255 0.45901639]
average cross-validation score: 0.4603870165024876
F1 train score: 0.7684548372306281
F1 test score: 0.4676258992805755


In [ ]:
#Naive Bayes
cross_val_scores = cross_val_score(MultinomialNB(), X_train_vector, y_train, cv=5, scoring="f1")
print(f"cross-validation scores:", cross_val_scores)
print(f"average cross-validation score:", cross_val_scores.mean())

justice_model = MultinomialNB()
justice_model.fit(X_train_vector, y_train)
y_pred_train = justice_model.predict(X_train_vector)
y_pred_test = justice_model.predict(X_test_vector)

f1_train = f1_score(y_train, y_pred_train)
f1_test = f1_score(y_test, y_pred_test)

print("F1 train score:", f1_train)
print("F1 test score:", f1_test)

cross-validation scores: [0.40816327 0.44393593 0.456621   0.4212963  0.41363636]
average cross-validation score: 0.42873057131568953
F1 train score: 0.7956502038966923
F1 test score: 0.43298969072164945
